In [1]:
'''Example script to generate text from Nietzsche's writings.
At least 20 epochs are required before the generated text
starts sounding coherent.
It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.
If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.
'''
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

Using TensorFlow backend.


606208/600901 [==============================] - 1s 1us/step
corpus length: 600893
total chars: 57
nb sequences: 200285
Vectorization...
Build model...
Epoch 1/60
200192/200285 [============================>.] - ETA: 0s - loss: 1.9886
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "ny of beings under whose hands, glances,"
ny of beings under whose hands, glances, and the more more the more of the more the common the world of the world of the more and the were and the fains of the asters of the respect of the were the soul of the world of the more the world of the sume as the more and the feeling of the self--a sumpor the more the more of the more and in the world of the something and the more the also the more the more and the more the more the world of t
----- diversity: 0.5
----- Generating with seed: "ny of beings under whose hands, glances,"
ny of beings under whose hands, glances, the heressele of intemplity constinented, as origin of the mor

spirited man, and an antire is gradation of the unfer to interpretation is find a philosophiclers and most contradiction that he are raterial the man is the mootes.--when the other when when when a still might of deepes of the inferent of the form has reality of beoted that here is also again and which he were all to the probably of 
----- diversity: 1.0
----- Generating with seed: "ur messmates--the
after-dinner nausea.

"
ur messmates--the
after-dinner nausea.

.

12o
.hatists, upinded understanted pails, as
unself.--certains and eye of this peomaniar is that the wet one season as the uppariately light man; thereleftred will we elevely how wills assimality. alwagtt the
ead? himself. "apteme is prefitu, been fellment of fundamencely mempley
aliok good fally and
probisped and evousishts, and a
forgure in simplly
has a proced-lication of not nestind, reason
----- diversity: 1.2
----- Generating with seed: "ur messmates--the
after-dinner nausea.

"
ur messmates--the
after-dinner nausea.


the scholars was conductibly sou, developalation of the
strong for it in fach
of his does to infulan hard, certain all perhaps
oft
mind, assers: are.
free case consequence absolvolx palles al
o amous
to recaleds
200285/200285 [==============================] - 49s 243us/step - loss: 1.4385
Epoch 8/60
200064/200285 [============================>.] - ETA: 0s - loss: 1.4237
----- Generating text after Epoch: 7
----- diversity: 0.2
----- Generating with seed: "has been invented:--such capacity has no"
has been invented:--such capacity has not the such a man of the such a such as the such and the serious and the such a such a surpement of the such a sult and such a sult and such a sult of the such a things and and something and always the such a sure and such a self-existence of the such a sult and the such a conduct of the such a sult and such and always the such a superious of the such a sense of the such a sult and and as the sense
----- diversity: 0.5
----- Generating with seed: "has be

/home/rkp/miniconda3/envs/nn/lib/python3.5/site-packages/ipykernel_launcher.py:63: RuntimeWarning: divide by zero encountered in log


...

nows the seems gaze "a, in a has ins arwars the itself their sensamily of the profraugh, so tholungs hey in question, "stupping that their fact in the effect is operince:
"the intention evidently incoiled themselviog do not pressord--my opposed childriness,  wouth abor every
element is ca kis one may rither in
----- diversity: 1.2
----- Generating with seed: "pposed
to the instinctive in any decisiv"
pposed
to the instinctive in any decisive religions.


1=h its
tank angained horlive.
this in decreases, he who ah-producions of unvaltofolent of
rule and traintic(). prelen,
is
indevegeitlings and heart,--heaving and play than the
nations",
i.

265. "that conceited, and gludy, usfuling tra! has been which is of
this sappease seoup origin too off another wall to mean that depeit of
last ttacted sidaon
men
sowor ultionally nancipsy bindf
200285/200285 [==============================] - 49s 244us/step - loss: 1.3221
Epoch 46/60
200192/200285 [============================>.] - ETA: 0s - loss:

much science in the state of the constitute the senses of the act of the state of the subjection of the state of the extraos: this even in the subjection of the state of the start that the soul of the state of the state of the soul and the subjection of the subjection of the state of the same strange the highest suffering of the state of the same or a states of the senses the state of the sense of the most resul
----- diversity: 0.5
----- Generating with seed: "uer's doctrine. there is
much science in"
uer's doctrine. there is
much science in itself, the secret in the secret from the same condition of their power man. it is the power of their present the present
sounds and self-extradation of religion and anftriod
of the conceptions, and the highest suffering and having the contramfery of the artist feelings that he is not the moral of the conditions of the things, the self-declang. it is an incort of the conditions of philosophical a
----- diversity: 1.0
----- Generating with seed: "u

soul and pisside just obutament, of fualless must above them custismaof and
excouropistics of a
becomed the influencely might of the order to all yet which is its
stricted at onstous, or by also of the powerful
consequence in all princee of these and la loggailius, connedence are dingy one in
----- diversity: 1.2
----- Generating with seed: " equally
erroneous): a thing cannot main"
 equally
erroneous): a thing cannot maintainamant
atompted,
only not
unperstitutius described is wide everything is acpual
rates.


1 
hos is the princimed taster, begaily down a
new emppoinisment as to be the dreams. perfects--la
shield, vervenally have a wangain, to any insolete at onde vicent, his
example, for itsibilic 

where cermitate comascious
most adolm, erroptessly
respected through
puritnet, the
most
sentime weakedful every m
200285/200285 [==============================] - 49s 243us/step - loss: 1.3192
Epoch 53/60
199936/200285 [============================>.] - ETA: 0s - loss: 1.3256
----- Gene

almost impalpable and such a more the more proposite of the strength, and the subtlers and the philosopher is the subtlere of the sense of the sense of the standard and and interprete to the sense of the properstful and and self-experience to the self-constant that is the suffering and and and and the world, and the properly and and strength and the conscience and and a man is not the sense of the subject of the a
----- diversity: 0.5
----- Generating with seed: "ell worth questioning,
almost impalpable"
ell worth questioning,
almost impalpable and more the highest friends: in the anmitted that has all the anti-to fastering of scientific with one be not yours ambited and not the philosopher persons of a communicative probably present ones, and in the philosopher of the world of an
the tertorn to an actions the unperfeque the despised, as a most conscience with a man who was a states, essential and the discoveler to the spirits--and fran
----- diversity: 1.0
----- Generating with seed: 

and of ear). this mind to ost alone--and in
they necessarilistous there are soul. it is respectous!--"covensions and consequently a dictuasation of theies, it gream
existh of
nature, there are
nurative even in all most richaby feeling of religious-faitherless which, there is also religion 
----- diversity: 1.2
----- Generating with seed: "ted here.]

191. the old theological pro"
ted here.]

191. the old theological pro-keeasiful
victern" is thic contrast, great cony any inable to confise
more character; he cad taste itself. a mas! such, as i make naturfaed ct inpullishment, if
sentiration whateveres itself, as reason.

88. 
pseverental
results caste--is fear of soul
him
has form that god)wers of knowledgeful scientif this last rigorfike, lestable menstm him, the will owiate a : is it evid in); a may respectinge
200285/200285 [==============================] - 51s 255us/step - loss: 1.3253
Epoch 60/60
200192/200285 [============================>.] - ETA: 0s - loss: 1.3247
----- Generat